# Tesserae v5 Demo

This demo will go over the basics of Tesserae v5 development up through February 5, 2019.

In [42]:
import json

from tesserae.db import TessMongoConnection
from tesserae.db.entities import Match, Text, Token, Unit
from tesserae.utils import TessFile
from tesserae.tokenizers import GreekTokenizer, LatinTokenizer
from tesserae.unitizer import Unitizer
from tesserae.matchers.sparse_encoding import SparseMatrixSearch
from tesserae.matchers.text_options import TextOptions

# Set up the connection and clean up the database
connection = TessMongoConnection('127.0.0.1', 27017, None, None, 'tesstest')

# Clean up the previous demo
connection.connection['features'].delete_many({})
connection.connection['matches'].delete_many({})
connection.connection['match_sets'].delete_many({})
connection.connection['texts'].delete_many({})
connection.connection['tokens'].delete_many({})
connection.connection['units'].delete_many({})

## Loading and Storing New Texts

The Tesserae database catalogs metadata, including the title, author, and year published, as well as integrity information like filepath, MD5 hash, and CTS URN.

We start by loading in some metadata from `text_metadata.json`.

In [43]:
with open('text_metadata.json', 'r') as f:
    text_meta = json.load(f)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
print('{}{}{}{}'.format('-----'.ljust(15), '------'.ljust(15), '--------'.ljust(15), '----'))
for t in text_meta:
    print('{}{}{}{}'.format(t['title'].ljust(15), t['author'].ljust(15), t['language'].ljust(15), str(t['year']).ljust(15)))

Title          Author         Language       Year
-----          ------         --------       ----
aeneid         vergil         latin          19             
de oratore     cicero         latin          38             
heracles       euripides      greek          -416           
epistles       plato          greek          -280           


Then insert the new texts with `TessMongoConnection.insert` after converting the raw JSON to Tesserae `Text` entities.

In [44]:
texts = []
for t in text_meta:
    texts.append(Text.json_decode(t))
result = connection.insert(texts)
print('Inserted {} texts.'.format(len(result.inserted_ids)))
print(result.inserted_ids)

Inserted 4 texts.
[ObjectId('651c51139c8f571a16438d42'), ObjectId('651c51139c8f571a16438d43'), ObjectId('651c51139c8f571a16438d44'), ObjectId('651c51139c8f571a16438d45')]


We can retrieve the inserted texts with `TessMongoConnection.find`. These texts will be converted to objects representing the database entries. The returned text list can be filtered by any valid field in the text database.

In [45]:
texts = connection.find('texts', _id=result.inserted_ids)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
for t in texts:
    print('{}{}{}{}'.format(t.title.ljust(15), t.author.ljust(15), t.language.ljust(15), t.year))

Title          Author         Language       Year
aeneid         vergil         latin          19
de oratore     cicero         latin          38
heracles       euripides      greek          -416
epistles       plato          greek          -280


## Loading .tess Files

Text metadata includes the path to the .tess file on the local filesystem. Using a Text retrieved from the database, the file can be loaded for further processing.

In [46]:
tessfile = TessFile(texts[0].path, metadata=texts[0])

print(tessfile.path)
print(len(tessfile))
print(tessfile[270])

la/vergil.aeneid.tess
9908
<verg. aen. 1.271>	transferet, et longam multa vi muniet Albam.



We can iterate through the file line-by-line.

In [47]:
lines = tessfile.readlines()
for i in range(10):
    print(next(lines))

<verg. aen. 1.1>	Arma virumque cano, Troiae qui primus ab oris

<verg. aen. 1.2>	Italiam, fato profugus, Laviniaque venit

<verg. aen. 1.3>	litora, multum ille et terris iactatus et alto

<verg. aen. 1.4>	vi superum saevae memorem Iunonis ob iram;

<verg. aen. 1.5>	multa quoque et bello passus, dum conderet urbem,

<verg. aen. 1.6>	inferretque deos Latio, genus unde Latinum,

<verg. aen. 1.7>	Albanique patres, atque altae moenia Romae.

<verg. aen. 1.8>	Musa, mihi causas memora, quo numine laeso,

<verg. aen. 1.9>	quidve dolens, regina deum tot volvere casus

<verg. aen. 1.10>	insignem pietate virum, tot adire labores



We can also iterate token-by-token.

In [48]:
tokens = tessfile.read_tokens()
for i in range(10):
    print(next(tokens))

Arma
virumque
cano,
Troiae
qui
primus
ab
oris
Italiam,
fato


## Tokenizing a Text

Texts can be tokenized with `tesserae.tokenizers` objects. These objects are designed to normalize and compute features for tokens of a specific language.

In [49]:
tokenizer = GreekTokenizer(connection) if tessfile.metadata.language == 'greek' else LatinTokenizer(connection)

tokens, tags, features = tokenizer.tokenize(tessfile.read(), text=tessfile.metadata)

print(len(tokens), len(tags), len(features))

print('{}{}{}{}'.format('Raw'.ljust(15), 'Normalized'.ljust(15), 'Lemmata'.ljust(20), 'Frequency'))
print('{}{}{}{}'.format('---'.ljust(15), '----------'.ljust(15), '-------'.ljust(20), '---------'))
for i in range(20):
    if len(tokens[i].features):
        print('{}{}{}{}'.format(tokens[i].display.ljust(15),
                              str(tokens[i].features['form'].token).ljust(20),
                              str(tokens[i].features['lemmata'][0].token).ljust(20),
                              list(tokens[i].features['form'].frequencies.values())[0]))

127526 9896 43223
Raw            Normalized     Lemmata             Frequency
---            ----------     -------             ---------
Arma           arma                arma                148
virumque       uirumque            uir                 4
cano           cano                canus               3
Troiae         troiae              troius              36
qui            qui                 quis                166
primus         primus              primus              34
ab             ab                  ab                  136
oris           oris                ora                 32
Italiam        italiam             italia              32
fato           fato                for                 7


Processed tokens can then be stored in and retrieved from the database, similar to text metadata.

In [50]:
result = connection.insert(features)
print('Inserted {} feature entities out of {}'.format(len(result.inserted_ids), len(features)))

Inserted 43223 feature entities out of 43223


## Unitizing a Text

Texts can be unitized into lines and phrases, and the intertext matches are found between units of text.


In [51]:
# Unitizing lines of a poem
unitizer = Unitizer()
lines, phrases = unitizer.unitize(tokens, tags, tessfile.metadata)

print('Lines\n-----')
for line in lines[:20]:
        print(''.join([str(line.tags), ': '] + [t['display']+' ' for t in line.tokens]))
        
print('\n\nPhrases\n-------')
for phrase in phrases[:20]:
        print(''.join([str(phrase.tags), ': '] + [t['display']+' ' for t in phrase.tokens]))

Lines
-----
['1.1']: Arma virumque cano Troiae qui primus ab oris 
['1.2']: Italiam fato profugus Laviniaque venit 
['1.3']: litora multum ille et terris iactatus et alto 
['1.4']: vi superum saevae memorem Iunonis ob iram 
['1.5']: multa quoque et bello passus dum conderet urbem 
['1.6']: inferretque deos Latio genus unde Latinum 
['1.7']: Albanique patres atque altae moenia Romae 
['1.8']: Musa mihi causas memora quo numine laeso 
['1.9']: quidve dolens regina deum tot volvere casus 
['1.10']: insignem pietate virum tot adire labores 
['1.11']: impulerit Tantaene animis caelestibus irae 
['1.12']: Urbs antiqua fuit Tyrii tenuere coloni 
['1.13']: Karthago Italiam contra Tiberinaque longe 
['1.14']: ostia dives opum studiisque asperrima belli 
['1.15']: quam Iuno fertur terris magis omnibus unam 
['1.16']: posthabita coluisse Samo hic illius arma 
['1.17']: hic currus fuit hoc regnum dea gentibus esse 
['1.18']: si qua fata sinant iam tum tenditque fovetque 
['1.19']: Progeniem sed en

In [52]:
# Unitizing phrases of a poem or prose
result = connection.insert(lines + phrases)
print('Inserted {} units out of {}.'.format(len(result.inserted_ids), len(lines + phrases)))


result = connection.insert(tokens)
print('Inserted {} tokens out of {}.'.format(len(result.inserted_ids), len(tokens)))

Inserted 14983 units out of 14983.
Inserted 127526 tokens out of 127526.


In [55]:
for text in texts[2:]:
    tessfile = TessFile(text.path, metadata=text)
    tokenizer = GreekTokenizer(connection) if tessfile.metadata.language == 'greek' else LatinTokenizer(connection)
    
    tokens, tags, features = tokenizer.tokenize(tessfile.read(), text=tessfile.metadata)
        
    result = connection.insert(features)

    unitizer = Unitizer()
    lines, phrases = unitizer.unitize(tokens, tags, tessfile.metadata)
    result = connection.insert(lines + phrases)
    
    result = connection.insert(tokens)


## Matching

Once the Texts, Tokens, and Units are in the database, we can then find intertext matches.

In [59]:
import time
matcher = SparseMatrixSearch(connection)
match_texts = [t for t in texts if t.language == 'greek']

start = time.time()
matches, match_set = matcher.match(TextOptions(match_texts[0],'phrase'),TextOptions(match_texts[1],'phrase'), 
                                   'form', distance_basis='span', stopwords=20, max_distance=10)
print("Completed matching in {0:.2f}s".format(time.time() - start))

matches.sort(key=lambda x: x.score, reverse=True)

# result = connection.insert(match_set)
# print('Inserted {} match set entities out of {}'.format(len(result.inserted_ids), 1))
result = connection.insert(matches)
print('Inserted {} match entities out of {}'.format(len(result.inserted_ids), len(matches)))

TypeError: match() missing 1 required positional argument: 'feature'

In [ ]:
matches = connection.aggregate('matches', [
    {'$match': {'match_set': match_set.id}},
    {'$sort': {'score': -1}},
    {'$limit': 20},
    {'$lookup': {
        'from': 'units',
        'let': {'m_units': '$units'},
        'pipeline': [
            {'$match': {'$expr': {'$in': ['$_id', '$$m_units']}}},
            {'$lookup': {
                'from': 'tokens',
                'localField': '_id',
                'foreignField': 'phrase',
                'as': 'tokens'
            }},
            {'$sort': {'index': 1}}
        ],
        'as': 'units'
    }},
    {'$lookup': {
        'from': 'tokens',
        'localField': 'tokens',
        'foreignField': '_id',
        'as': 'tokens'
    }},
    {'$project': {
        'units': True,
        'score': True,
        'tokens': '$tokens.feature_set'
    }},
    {'$lookup': {
        'from': 'feature_sets',
        'localField': 'tokens',
        'foreignField': '_id',
        'as': 'tokens'
    }}
])

print('\n')
print('{}{}'.format('Score'.ljust(15), 'Match Tokens'.ljust(15)))
print('{}{}'.format('-----'.ljust(15), '------------'.ljust(15)))
for m in matches:
    print('{}{}'.format(('%.3f'%(m.score)).ljust(15), ', '.join(list(set([t['form'] for t in m.tokens])))))
    print('{} {} {}: {}'.format(match_texts[0].author, match_texts[0].title, m.units[0]['tags'], ''.join([t['display'] for t in m.units[0]['tokens']])))
    print('{} {} {}: {}'.format(match_texts[1].author, match_texts[1].title, m.units[1]['tags'], ''.join([t['display'] for t in m.units[1]['tokens']])))
    print('\n')